In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

In [17]:
# Print current directory and contents
print("Current working directory:", os.getcwd())
print("Contents of current directory:", os.listdir())

Current working directory: c:\Users\Dell\Desktop\Task2Code
Contents of current directory: ['Codefile.ipynb', 'Test', 'Train', 'Validation']


In [21]:
base_dir = r'c:\Users\Dell\Desktop\Task2Code'

In [18]:
# Create dataset paths 
train_path = os.path.join('Train', 'Train')
valid_path = os.path.join('Validation', 'Validation')
test_path = os.path.join('Test', 'Test')

In [19]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Smaller size for baseline model
    transforms.ToTensor()
])

In [22]:
# Create dataset paths
train_path = os.path.join(base_dir, 'Train', 'Train')
valid_path = os.path.join(base_dir, 'Validation', 'Validation')
test_path = os.path.join(base_dir, 'Test', 'Test')

In [24]:

# Create datasets
train_dataset = datasets.ImageFolder(train_path, transform=transform)
valid_dataset = datasets.ImageFolder(valid_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)


In [25]:
# Print dataset information
print(f'Number of training images: {len(train_dataset)}')
print(f'Number of validation images: {len(valid_dataset)}')
print(f'Number of test images: {len(test_dataset)}')
print(f'Classes: {train_dataset.classes}')


Number of training images: 1322
Number of validation images: 60
Number of test images: 150
Classes: ['Healthy', 'Powdery', 'Rust']


In [26]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)